# Chapter 7. 텍스트 문서의 범주화 - (3) 리뷰 감성 분류기 구현


- 이제 앞에서 구현한 CNN 문서 모델을 훈련해서 감성 분류기를 구축해 보자
- 캐글에서 아마존 감성 분석 리뷰 데이터 세트를 다운로드 받아 압축해제하여 저장한다. (train.ft.txt와 test.ft.txt 두 파일 모두 다운)
    - 다운로드 url
        - https://www.kaggle.com/bittlingmayer/amazonreviews
    - 저장경로
        - train.ft.txt -> data/amazonreviews/train.ft/train.ft.txt
        - test.ft.txt -> data/amazonreviews/test.ft/test.ft.txt

In [1]:
# !kaggle datasets download -d bittlingmayer/amazonreviews

amazonreviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [2]:
# !unzip amazonreviews.zip -d ./data/amazonreviews

Archive:  amazonreviews.zip
  inflating: ./data/amazonreviews/test.ft.txt.bz2  
  inflating: ./data/amazonreviews/train.ft.txt.bz2  


In [3]:
# !bzip2 ./data/amazonreviews/test.ft.txt.bz2 -d 
# !bzip2 ./data/amazonreviews/train.ft.txt.bz2 -d 

In [5]:
import os
import config
from dataloader.loader import Loader
from preprocessing.utils import Preprocess, remove_empty_docs
from dataloader.embeddings import GloVe
from model.cnn_document_model import DocumentModel, TrainingParameters
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np

## 아마존 리뷰 데이터 로드

- 아마존 리뷰 데이터를 로드한다. (data/amazonreviews 경로)
    - 360만개의 훈련 샘플과 40만개의 테스트 샘플이 있다. train 데이터셋은 랜덤으로 20만개만 추출하여 사용한다
    - <b>\__label__1</b>은 별점 1-2점을 매긴 리뷰에 해당, <b>\__label__2</b>는 별점 4-5점을 매긴 리뷰에 해당한다
    - 별점 3점의 리뷰, 즉, 중립적인 감성을 가진 리뷰는 이 데이터 세트에 포함되지 않았다
    - 원본 데이터 예시
````
__label__<X> <summary/title>: <Review Text>
Example:
__label__2 Good Movie: Awesome.... simply awesome. I couldn't put this down
and laughed, smiled, and even got tears! A brand new favorite author.
```

- 아마존 리뷰 데이터를 데이터프레임으로 변환한다
    - sentiment 칼럼에 0(부정) 또는 1(긍정) 값을 입력
    - 데이터프레임 예시
```
index   review                                              sentiment
0       Stuning even for the non-gamer . This sound t...    1
1       The best soundtrack ever to anything. . I'm r...    1
2       Amazing! . This soundtrack is my favorite mus...    1
3       Excellent Soundtrack: I truly like this soundt...   1
4       Remember, Pull Your Jaw Off The Floor After He...   1
```

In [7]:
# dataloader/loader.py 의 Loader.load_amazon_reviews 참고

# 아마존 리뷰 데이터를 로드하여 데이터프레임으로 변환한다
train_df = Loader.load_amazon_reviews('train')
print(f'train_df.shape : {train_df.shape}')

test_df = Loader.load_amazon_reviews('test')
print(f'test_df.shape : {test_df.shape}')

train_df.shape : (3600000, 2)
test_df.shape : (400000, 2)


In [8]:
# 학습셋에서 랜덤으로 20만개만 추출하여 feature 추출에 사용한다
dataset = train_df.sample(n=200000, random_state=42)
dataset.sentiment.value_counts()

sentiment
1    100020
0     99980
Name: count, dtype: int64

In [9]:
train_df.head()

,review,sentiment
0,Stuning even for the non-gamer . This sound t...,1
1,The best soundtrack ever to anything. . I'm r...,1
2,Amazing! . This soundtrack is my favorite mus...,1
3,Excellent Soundtrack: I truly like this soundt...,1
4,"Remember, Pull Your Jaw Off The Floor After He...",1


## 인덱스 시퀀스 변환

In [10]:
# 추출한 20만개 데이터 샘플에서 review, sentiment 칼럼 값들 추출
corpus = dataset['review'].values
target = dataset['sentiment'].values
print(f'corpus.shape : {corpus.shape}')
print(f'target.shape : {target.shape}')

# 유효하지 않은 값 제거 (비어있거나 길이가 30 이하인 경우 제거)
corpus, target = remove_empty_docs(corpus, target)
print('=== after remove_empty_docs ===')
print(f'corpus size : {len(corpus)}')
print(f'target size : {len(target)}')

corpus.shape : (200000,)
target.shape : (200000,)
=== after remove_empty_docs ===
corpus size : 200000
target size : 200000


In [11]:
# 20만개 데이터 샘플에 대해 인덱스 사전 구축 및 인덱스 시퀀스 변환
preprocessor = Preprocess(corpus=corpus)
corpus_to_seq = preprocessor.fit()

Found 43195 unique tokens.
All documents processed.ocessed.

In [12]:
print(f'corpus_to_seq size : {len(corpus_to_seq)}')
print(f'corpus_to_seq[0] size : {len(corpus_to_seq[0])}')
print(f'corpus_to_seq[0] :')
print(corpus_to_seq[0])

corpus_to_seq size : 200000
corpus_to_seq[0] size : 300
corpus_to_seq[0] :
[ 2  3  4  5  6  7  8  9  7 10 11 12 13 14 15 16 17 18 19 20 21  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0 22 23 24 25 26 27 28 29  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 16 30 31 32 33 34
 17 30 35 36 37 14  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 38 39 40 41 42 37 16 43 44 45 46 17 37 47 48 37 49  0  0  0  0  0  0  0
  0  0  0  0  0  0 18 19 20 30 50 51 52 17 53 54 46 55 56 36 57  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0 37 58 59  8 60 39 61 62 63 64 65 59
 66 41 67 68 28 69 17 70 71 72  0  0  0  0  0  0  0  0 39 61 73 74 75 76
  4  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0]


In [13]:
# 인덱싱되기 전 원본 문서
corpus[0]

'Expensive Junk: This product consists of a piece of thin flexible insulating material, adhesive backed velcro and white electrical tape.Problems . 1. Instructions are three pictures with little more information.2. Velcro was all crumpled as received and was stronger than the adhesive. When i tried to disengage the velcro both pieces came off and the paint from the ceiling.3. White electrical tape was horrible... cheap, narrow and it fell off in less than 1 hour.4. The price is a ripoff.I am building my own which is easier to use, cheaper, more attractive, and higher r-value. I am surprised Amazon even lists this junk.'

In [14]:
# 테스트셋(test_df) 40만건 리뷰에서 review, sentiment 칼럼 값 추출
holdout_corpus = test_df['review'].values
holdout_target = test_df['sentiment'].values
print(f'holdout_corpus.shape : {holdout_corpus.shape}')
print(f'holdout_target.shape : {holdout_target.shape}')

# 유효하지 않은 값 제거 (비어있거나 길이가 30 이하인 경우 제거)
holdout_corpus, holdout_target = remove_empty_docs(holdout_corpus, holdout_target)
print('=== after remove_empty_docs ===')
print(f'holdout_corpus size : {len(holdout_corpus)}')
print(f'holdout_target size : {len(holdout_target)}')

holdout_corpus.shape : (400000,)
holdout_target.shape : (400000,)
=== after remove_empty_docs ===
holdout_corpus size : 400000
holdout_target size : 400000


In [15]:
# 테스트셋을 인덱스 시퀀스로 변환 (위에서 생성한 인덱스 사전 그대로 사용)
holdout_corpus_to_seq = preprocessor.transform(holdout_corpus)

All documents processed.ocessed.

In [16]:
print(f'holdout_corpus_to_seq size : {len(holdout_corpus_to_seq)}')
print(f'holdout_corpus_to_seq[0] size : {len(holdout_corpus_to_seq[0])}')
print(f'holdout_corpus_to_seq[0] :')
print(holdout_corpus_to_seq[0])

holdout_corpus_to_seq size : 400000
holdout_corpus_to_seq[0] size : 300
holdout_corpus_to_seq[0] :
[  335   336     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0    63  4565  6750   132   120     7
    37   335  2779     7   244  3736     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
    39    91   569    41     4   336    83   765    17    39   670  1043
    53     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0    38    39   449    55     8   238
  9021    53   262   214  1112   131     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     8   178  9021   184 19876   118  5560    55    37  1317     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0 

## 임베딩 초기화

In [17]:
# 인덱싱된 텍스트 데이터를 GloVe로 임베딩 초기화.
# glove.6B.50d.txt에 없는 단어는 OOV..txt에 write한다
# word_index는 {'expensive': 2, 'junk': 3, 'this': 4, ...} 형태의 인덱싱 사전
glove = GloVe(50)
initial_embeddings = glove.get_embedding(preprocessor.word_index)  

Reading 50 dim GloVe vectors
Found 400000 word vectors.
words not found in embeddings: 2582


In [18]:
# 인덱스 사전의 단어 수
len(preprocessor.word_index)

43195

In [19]:
# GloVe로 임베딩 초기화된 행렬. 벡터 개수는 word_index 인덱스 사전의 단어 + 2, 차원 수는 50이다
initial_embeddings.shape

(43197, 50)

glove6B.50d의 단어 수는 40만개이며, 이 중 아마존 리뷰 데이터 속 4만 3천여개 단어에 대한 임베딩 행렬을 생성하였다

## CNN 감성분석 모델 생성

In [20]:
# model/cnn_document_model.py의 DocumentModel 클래스 참조

# CNN 기반 문서 분류 모델 인스턴스 생성. 위에서 GloVe로 만든 임베딩 행렬을 임베딩 초깃값으로 사용한다
amazon_review_model = DocumentModel(vocab_size=preprocessor.get_vocab_size(),
                                    word_index=preprocessor.word_index,
                                    num_sentences=Preprocess.NUM_SENTENCES,
                                    embedding_weights=initial_embeddings,
                                    conv_activation='tanh',
                                    hidden_dims=64,
                                    input_dropout=0.40,
                                    hidden_gaussian_noise_sd=0.5)

Vocab Size = 43197  and the index of vocabulary words passed has 43195 words


## 모델 학습

In [21]:
# 학습된 모델을 저장할 디렉토리 생성
if not os.path.exists(os.path.join(config.MODEL_DIR, 'amazonreviews')):
    os.makedirs(os.path.join(config.MODEL_DIR, 'amazonreviews'))

# 학습 파라미터 저장 클래스
train_params = TrainingParameters('model_with_tanh_activation', 
                                  model_file_path = config.MODEL_DIR + '/amazonreviews/model_06.hdf5',
                                  model_hyper_parameters = config.MODEL_DIR + '/amazonreviews/model_06.json',
                                  model_train_parameters = config.MODEL_DIR + '/amazonreviews/model_06_meta.json',
                                  num_epochs=35)

# 모델 컴파일
amazon_review_model.get_classification_model().compile(loss="binary_crossentropy", 
                                                       optimizer=train_params.optimizer,
                                                       metrics=["accuracy"])

# callback (1) - 자동저장 체크포인트
checkpointer = ModelCheckpoint(filepath=train_params.model_file_path,
                               verbose=1,
                               save_best_only=True,
                               save_weights_only=True)

# callback (2) - 조기종료
early_stop = EarlyStopping(patience=2)

# 모델에 입력할 학습데이터, 테스트데이터 (인덱스 값들의 시퀀스로 변환된 값)
x_train = np.array(corpus_to_seq)
y_train = np.array(target)
x_test = np.array(holdout_corpus_to_seq)
y_test = np.array(holdout_target)
print(f'x_train.shape : {x_train.shape}')
print(f'y_train.shape : {y_train.shape}')
print(f'x_test.shape : {x_test.shape}')
print(f'y_test.shape : {y_test.shape}')

# 모델 훈련 시작
amazon_review_model.get_classification_model().fit(x_train,
                                                   y_train, 
                                                   batch_size=train_params.batch_size, 
                                                   epochs=train_params.num_epochs,  # 35 epochs
                                                   verbose=2,
                                                   validation_split=train_params.validation_split, # 5%
                                                   callbacks=[checkpointer])

# 모델 저장
amazon_review_model._save_model(train_params.model_hyper_parameters)

x_train.shape : (200000, 300)
y_train.shape : (200000,)
x_test.shape : (400000, 300)
y_test.shape : (400000,)
Epoch 1/35

Epoch 1: val_loss improved from inf to 0.68892, saving model to ./checkpoint/amazonreviews/model_06.hdf5
2969/2969 - 77s - loss: 0.6941 - accuracy: 0.5088 - val_loss: 0.6889 - val_accuracy: 0.5062 - 77s/epoch - 26ms/step
Epoch 2/35

Epoch 2: val_loss improved from 0.68892 to 0.68468, saving model to ./checkpoint/amazonreviews/model_06.hdf5
2969/2969 - 78s - loss: 0.6890 - accuracy: 0.5159 - val_loss: 0.6847 - val_accuracy: 0.5254 - 78s/epoch - 26ms/step
Epoch 3/35

Epoch 3: val_loss improved from 0.68468 to 0.68351, saving model to ./checkpoint/amazonreviews/model_06.hdf5
2969/2969 - 79s - loss: 0.6868 - accuracy: 0.5197 - val_loss: 0.6835 - val_accuracy: 0.5250 - 79s/epoch - 27ms/step
Epoch 4/35

Epoch 4: val_loss improved from 0.68351 to 0.68340, saving model to ./checkpoint/amazonreviews/model_06.hdf5
2969/2969 - 78s - loss: 0.6859 - accuracy: 0.5218 - val_loss: 

In [22]:
# 모델 평가 - 테스트 데이터셋으로 수행
amazon_review_model.get_classification_model().evaluate(x_test,
                                                        y_test, 
                                                        train_params.batch_size*10,
                                                        verbose=2)

625/625 - 17s - loss: 0.6862 - accuracy: 0.5259 - 17s/epoch - 28ms/step


[0.6862403750419617, 0.5258899927139282]

## 가장 많이 변경된 임베딩은 무엇일까?

In [23]:
learned_embeddings = amazon_review_model.get_classification_model().get_layer('imdb_embedding').get_weights()[0]

embd_change = {}
for word, i in preprocessor.word_index.items():
    # Frobenium norm (Euclidean norm) 계
    embd_change[word] = np.linalg.norm(initial_embeddings[i]-learned_embeddings[i])
embd_change = sorted(embd_change.items(), key=lambda x: x[1], reverse=True)
embd_change[0:20]

[('nothing', 5.898498454582604),
 ('never', 5.838235222411407),
 ('not', 5.837764425357621),
 ('money', 5.567684147550348),
 ('instead', 5.473179924519233),
 ('better', 5.338847887023589),
 ('bad', 5.176505421221354),
 ('waste', 4.991878514338513),
 ('guess', 4.919912188564288),
 ('back', 4.898684547565275),
 ('what', 4.791495606601605),
 ('return', 4.621494796555242),
 ('same', 4.567734369178881),
 ('worst', 4.561360698176675),
 ('something', 4.4871946257678745),
 ('poor', 4.479186207022457),
 ('sorry', 4.285149087759856),
 ('tried', 4.224736281244272),
 ('no', 4.189658446760836),
 ('boring', 4.187275067385223)]